In [140]:
import torch
import pytorch_lightning as L
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [141]:
import sys
try:
    del sys.modules['data_loading']
    del sys.modules['transformer_predictor']
except:
    pass

from data_loading import load_data, get_loaders, BracketDataset
from transformer_predictor import TransformerPredictor

In [142]:
dataset = load_data('../Data/train-CoT.csv')

  0%|          | 0/40000 [00:00<?, ?it/s]

In [143]:
dataset[0][0].shape, dataset[0][1].shape

(torch.Size([728]), torch.Size([728]))

In [144]:
BATCH_SIZE = 64
train_loader, val_loader, test_loader, train_data, val_data, test_data = get_loaders(dataset, batch_size=BATCH_SIZE, return_data=True)

In [145]:
print(len(train_data), len(val_data), len(test_data), len(dataset))

28000 4000 8000 40000


In [146]:
def get_name_from_config(config):
    """
    Convert a config dict to the string under which the corresponding
    models and datasets will be saved.
    """
    return f'd_model={config["model_dim"]}-nhead={config["num_heads"]}-nlayers={config["num_layers"]}'

In [147]:
config = {
    'model_dim': 128,
    'num_heads': 8,
    'num_layers': 3,
    'lr': 1e-3
}

In [148]:
model = TransformerPredictor(
    input_dim=6,
    model_dim=config['model_dim'],
    num_heads=config['num_heads'],
    num_layers=config['num_layers'],
    lr=config['lr'],
)
name = get_name_from_config(config)
early_stopping =  EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint(monitor='val_loss', save_top_k=1, dirpath='models/', filename=name)
trainer = L.Trainer(max_epochs=50, devices=1, callbacks=[early_stopping, model_checkpoint])

/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home2/adyansh/dinner_pool/rsai/lib/python3.10/site- ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [149]:
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | TransformerLM    | 300 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
300 K     Trainable params
0         Non-trainable params
300 K     Total params
1.201     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [150]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.030484776943922043    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.030484776943922043}]

In [170]:
seq = '(())()&'
ctoi = {c: i for i, c in enumerate('()&YNP')}
itoc = {i: c for c, i in ctoi.items()}

src = torch.Tensor([ctoi[c] for c in seq]).long().unsqueeze(0)

out = model.model.complete_sequence(src)

for i in out[0]:
    print(itoc[i.item()], end='')

tensor([[4]])
Done!
(())()&N